# **Minerando Dados - A maior comunidade de Data Science do Brasil**
www.minerandodados.com.br

## Similaridade de Strings

Aplicações:
    * Data Cleaning
    * Correção de digitação
    * Tradução de idiomas

FuzzyWuzzy - Levenshtein_distance - https://en.wikipedia.org/wiki/Levenshtein_distance

In [1]:
!pip install fuzzywuzzy

In [2]:
!pip install fuzzywuzzy[speedup]

     |████████████████████████████████| 51kB 2.5MB/s 
  Created wheel for python-levenshtein: filename=python_Levenshtein-0.12.0-cp36-cp36m-linux_x86_64.whl size=144788 sha256=669ee678c51c62af58e085aeca2d201d9fa25aa20377448388468d799c058482
  Stored in directory: /root/.cache/pip/wheels/de/c2/93/660fd5f7559049268ad2dc6d81c4e39e9e36518766eaf7e342
Successfully built python-levenshtein


**Importa a biblitoteca**

In [3]:
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

**Aplicando a fuzzywuzzy em duas strings**

In [4]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovascular'
fuzz.ratio(s1,s2)

100

In [5]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovasculhar'
fuzz.ratio(s1,s2)

98

**Letras maiusculas e minúsculas**

In [6]:
s1 = 'Doença Cardiovascular'
s2 = 'doença Cardiovascular'
fuzz.ratio(s1,s2)

95

** Pontuação ou outros caracteres influenciam no score**

In [7]:
s1 = 'Doença Cardiovascular'
s2 = 'Doença Cardiovascular!!'
fuzz.ratio(s1,s2)

95

## Similaridade Parcial

* Similaridade parcial busca apenas a string em questão e descarta o resto.
* Extremamente útil para trabalhar com dados coletados da web.

In [8]:
# Consultando o score usando o método ratio
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovascular#$#%#ˆˆˆˆˆ!!'
fuzz.ratio(s1,s2)

68

In [9]:
# Consultando o score usando o método partial
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovascular#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1,s2)

100

In [10]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = '###$$%$!Doença Cardiovasculhar#$#%#ˆˆˆˆˆ!!'
fuzz.partial_ratio(s1,s2)

95

** Ordem de caracteres diferentes?**

In [11]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.partial_ratio(s1,s2)

67

* Função **partial_token_sort_ratio()** separa os tokens por espaço e ordena por ordem alfabética.
* Coloca as strings em letras minúsculas.
* Considera apenas as strings consultadas.

In [12]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'Cardiovascular Doença'
fuzz.partial_token_sort_ratio(s1,s2)

100

In [13]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = 'cardiovascular doença'
fuzz.partial_token_sort_ratio(s1,s2)

100

In [14]:
# Consultando o score usando o método partial
# alteração nas strings
s1 = 'Doença Cardiovascular'
s2 = '%%%%%cardiovascular doença&&&&****@@@'
fuzz.partial_token_sort_ratio(s1,s2)

100

## Processando uma Lista de Strings

* Aplicar o fuzzywuzzy para corrigir strings em uma base de dados

In [15]:
from fuzzywuzzy import process

** Cria lista de strings**

In [16]:
lista = ['Doença Cardiovascular.','doença cardiovascular!!', 'Doenca Cardiovascular', 'Doenc. Cardio']

In [17]:
lista

['Doença Cardiovascular.',
 'doença cardiovascular!!',
 'Doenca Cardiovascular',
 'Doenc. Cardio']

** Extrai os scores de similaridades com uma string em questão**

In [18]:
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio)

[('Doença Cardiovascular.', 100),
 ('doença cardiovascular!!', 100),
 ('Doenca Cardiovascular', 95),
 ('Doenc. Cardio', 85)]

** Limitando o retorno**

In [19]:
process.extract('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio,limit=2)

[('Doença Cardiovascular.', 100), ('doença cardiovascular!!', 100)]

** Retorna apenas uma string com um score acima de 95**

In [20]:
# Método extractOne
process.extractOne('Doença Cardiovascular', lista, scorer=fuzz.partial_ratio, score_cutoff=95)

('Doença Cardiovascular.', 100)

## Data Cleaning em um DataFrame

* Aplicar o fuzzywuzzy em uma base de dados
* Medir a similaridade de strings e fazer **Data Cleaning**

In [21]:
import pandas as pd
from collections import OrderedDict
data = OrderedDict(
    {
        'codigo_produto': [10 , 11, 12, 13, 14],
        'descrição': ['iphone 6ss', 'iphone 6s', 'iphoni 6s', 'ipone 6s','Iphone 6s,,,,']
        
    })

In [22]:
data

OrderedDict([('codigo_produto', [10, 11, 12, 13, 14]),
             ('descrição',
              ['iphone 6ss',
               'iphone 6s',
               'iphoni 6s',
               'ipone 6s',
               'Iphone 6s,,,,'])])

In [23]:
dataset = pd.DataFrame(data)

In [24]:
dataset

,codigo_produto,descrição
0,10,iphone 6ss
1,11,iphone 6s
2,12,iphoni 6s
3,13,ipone 6s
4,14,"Iphone 6s,,,,"


In [25]:
process.extractOne('Iphone 6s', choices=dataset.descrição, scorer=fuzz.ratio, score_cutoff=95)

('iphone 6s', 100, 1)

** Função que aplica o Fuzzy**

In [26]:
def AplicaFuzzy(query, dados, metodo_ratio, score_corte):
    return process.extractOne(query, choices=dados, scorer=metodo_ratio, score_cutoff=score_corte)

** Testa a função**

In [31]:
AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 90)

('iphone 6s', 100, 1)

** Criando uma nova coluna no Dataset a partir das string similares**

In [32]:
dataset['descrição2'] = AplicaFuzzy('Iphone 6s', dataset.descrição, fuzz.ratio, 95)[0]

In [33]:
dataset

,codigo_produto,descrição,descrição2
0,10,iphone 6ss,iphone 6s
1,11,iphone 6s,iphone 6s
2,12,iphoni 6s,iphone 6s
3,13,ipone 6s,iphone 6s
4,14,"Iphone 6s,,,,",iphone 6s
